## Import Dependencies

In [ ]:
import numpy as np
from Scripts import VoxelizeData
from Scripts import VoxNet
import torch

## Voxelize Necessary Datasets

### Voxelize ModelNet40 Dataset

In [ ]:
path = './Data/ModelNet40/'
dataset = 'ModelNet40'

# vox = VoxelizeData.Voxelize(path, dataset, system='Windows', save=True, save_path=path, overwrite=False)

### Test Voxelized ModelNet40 Data

In [ ]:
# Load training data
data = np.load('Data/ModelNet40/ModelNet40Train.npz', allow_pickle=True)

# Parser cuts off anything with an underscore. This array is modified to match
modelnet_labels = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower',
    'glass', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night', 'person', 'piano', 'plant', 'radio', 'range', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv', 'vase',
    'wardrobe', 'xbox'
]

train_data = data['data']
train_labels = data['labels']

print(train_data.shape)
print(train_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(modelnet_labels.index(i)))

train_data = torch.utils.data.TensorDataset(torch.FloatTensor(train_data),torch.LongTensor(label_as_int))

# vox.render(train_data[0])

# Load test data
data = np.load('Data/ModelNet40/ModelNet40Test.npz', allow_pickle=True)

test_data = data['data']
test_labels = data['labels']

print(test_data.shape)
print(test_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(modelnet_labels.index(i)))

test_data = torch.utils.data.TensorDataset(torch.FloatTensor(test_data),torch.LongTensor(label_as_int))

# vox.render(test_data[0])

# Hyperparameters
batch_size = 32
num_classes = 40
learning_rate = 0.01 # Decreases by a factor of 10 every 80000 batches
momentum = 0.9
weight_decay = 0.001
epochs = 1

# Load data in batch sizes
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)

model = VoxNet.VoxNet(num_classes)
model = model.float()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    print("epoch:", epoch)
    for (vox_grids, vox_labels) in train_loader:
        # print(len(vox_grids))
        if (len(vox_grids) < batch_size):
            continue

        output = model(vox_grids)
        loss = criterion(output, vox_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Test VoxNet after training
    with torch.no_grad():
        num_correct = 0
        total = 0

        for (grids, labels) in test_loader:
            if (len(grids) < batch_size):
                continue
            
            output = model(grids)

            _, predictions = torch.max(output, 1)

            num_correct += (predictions == labels).sum().item()
            total += labels.shape[0]
        
        accuracy = (num_correct / total) * 100.0

        print(accuracy)



### Voxelize Toy4K Dataset

In [ ]:
path = 'Data/toys4k_obj_files/'
dataset = 'Toys4K'

vox = VoxelizeData.Voxelize(path, dataset, system='Unix', render=True, save=True, save_path=path, overwrite=False)